In [1]:
from langchain_groq import ChatGroq

llm = ChatGroq(
    temperature=0, 
    groq_api_key='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', 
    model_name="llama-3.1-70b-versatile"
)

response = llm.invoke("The first person to land on moon was ...")
print(response.content)

The first person to land on the moon was Neil Armstrong. He stepped onto the lunar surface on July 20, 1969, as part of the Apollo 11 mission.


In [2]:
from langchain_community.document_loaders import WebBaseLoader

loader = WebBaseLoader("https://jobs.lever.co/mediaocean/2ae6442d-c1bc-4dec-ad90-6bc3c28354c9/")
page_data = loader.load().pop().page_content
print(page_data)

USER_AGENT environment variable not set, consider setting it to identify your requests.


Mediaocean - Software Engineer - Engineering Support (L2)Software Engineer - Engineering Support (L2)PuneAd Infrastructure – Global Platform Engineering /Full-time /HybridApply for this jobMediaocean is powering the future of the advertising ecosystem with technology that empowers brands and agencies to deliver impactful omnichannel marketing experiences. With over $200 billion in annualized ad spend running through its software products, Mediaocean deploys AI and automation to optimize investments and outcomes. The company's advertising infrastructure and ad tech tools are used by more than 100,000 people across the globe. Mediaocean owns and operates Prisma, the industry's trusted system of record for media management and finance, Flashtalking, the world's largest independent ad server and creative personalization platform, as well as Protected Media, the MRC-accredited ad verification solution for brand safety and fraud detection. Visit www.mediaocean.com for more information. What 

In [4]:
from langchain_core.prompts import PromptTemplate

prompt_extract = PromptTemplate.from_template(
        """
        ### SCRAPED TEXT FROM WEBSITE:
        {page_data}
        ### INSTRUCTION:
        The scraped text is from the career's page of a website.
        Your job is to extract the job postings and return them in JSON format containing the 
        following keys: `role`, `experience`, `skills` and `description`.
        Only return the valid JSON.
        ### VALID JSON (NO PREAMBLE):    
        """
)

chain_extract = prompt_extract | llm 
res = chain_extract.invoke(input={'page_data':page_data})
res


AIMessage(content='```\n{\n  "role": "Software Engineer - Engineering Support (L2)",\n  "experience": "Relevant work experience supporting an enterprise application",\n  "skills": [\n    "Hands-on experience in writing basic to medium complex SQL queries for Oracle/MSSQL database",\n    "Basic understanding of Unix Shell scripting",\n    "Experience in debugging java/.net web application and decipher log files to identify root cause",\n    "Above 60%, Engineering or graduation in IT/Comp Sc",\n    "Very strong communication skill both in written and spoken"\n  ],\n  "description": "Working to troubleshoot and determine root cause for technical escalations on an enterprise web application. Troubleshooting issues by going through the log files with a thorough attention to detail. Provide assistance on medium to high level complexity support inquiries. Responding quickly and intelligently to production issues and facilitating communication between L1 and Development teams spread across Au

In [5]:
type(res.content)

str

In [6]:
from langchain_core.output_parsers import JsonOutputParser

json_parser = JsonOutputParser()
json_res = json_parser.parse(res.content)
json_res

{'role': 'Software Engineer - Engineering Support (L2)',
 'experience': 'Relevant work experience supporting an enterprise application',
 'skills': ['Hands-on experience in writing basic to medium complex SQL queries for Oracle/MSSQL database',
  'Basic understanding of Unix Shell scripting',
  'Experience in debugging java/.net web application and decipher log files to identify root cause',
  'Above 60%, Engineering or graduation in IT/Comp Sc',
  'Very strong communication skill both in written and spoken'],
 'description': 'Working to troubleshoot and determine root cause for technical escalations on an enterprise web application. Troubleshooting issues by going through the log files with a thorough attention to detail. Provide assistance on medium to high level complexity support inquiries. Responding quickly and intelligently to production issues and facilitating communication between L1 and Development teams spread across Australia & US.'}

In [7]:
type(json_res)

dict

In [8]:
import pandas as pd

df = pd.read_csv("my_portfolio.csv")
df

,Techstack,Links
0,"React, Node.js, MongoDB",https://example.com/react-portfolio
1,"Angular,.NET, SQL Server",https://example.com/angular-portfolio
2,"Vue.js, Ruby on Rails, PostgreSQL",https://example.com/vue-portfolio
3,"Python, Django, MySQL",https://example.com/python-portfolio
4,"Java, Spring Boot, Oracle",https://example.com/java-portfolio
5,"Flutter, Firebase, GraphQL",https://example.com/flutter-portfolio
6,"WordPress, PHP, MySQL",https://example.com/wordpress-portfolio
7,"Magento, PHP, MySQL",https://example.com/magento-portfolio
8,"React Native, Node.js, MongoDB",https://example.com/react-native-portfolio
9,"iOS, Swift, Core Data",https://example.com/ios-portfolio


In [9]:
import uuid
import chromadb

client = chromadb.PersistentClient('vectorstore')
collection = client.get_or_create_collection(name="portfolio")

if not collection.count():
    for _, row in df.iterrows():
        collection.add(documents=row["Techstack"],
                       metadatas={"links": row["Links"]},
                       ids=[str(uuid.uuid4())])

In [12]:
job = json_res
job['skills']

['Hands-on experience in writing basic to medium complex SQL queries for Oracle/MSSQL database',
 'Basic understanding of Unix Shell scripting',
 'Experience in debugging java/.net web application and decipher log files to identify root cause',
 'Above 60%, Engineering or graduation in IT/Comp Sc',
 'Very strong communication skill both in written and spoken']

In [13]:
job

{'role': 'Software Engineer - Engineering Support (L2)',
 'experience': 'Relevant work experience supporting an enterprise application',
 'skills': ['Hands-on experience in writing basic to medium complex SQL queries for Oracle/MSSQL database',
  'Basic understanding of Unix Shell scripting',
  'Experience in debugging java/.net web application and decipher log files to identify root cause',
  'Above 60%, Engineering or graduation in IT/Comp Sc',
  'Very strong communication skill both in written and spoken'],
 'description': 'Working to troubleshoot and determine root cause for technical escalations on an enterprise web application. Troubleshooting issues by going through the log files with a thorough attention to detail. Provide assistance on medium to high level complexity support inquiries. Responding quickly and intelligently to production issues and facilitating communication between L1 and Development teams spread across Australia & US.'}

In [14]:
links = collection.query(query_texts=job['skills'], n_results=2).get('metadatas', [])
links

[[{'links': 'https://example.com/magento-portfolio'},
  {'links': 'https://example.com/java-portfolio'}],
 [{'links': 'https://example.com/magento-portfolio'},
  {'links': 'https://example.com/devops-portfolio'}],
 [{'links': 'https://example.com/java-portfolio'},
  {'links': 'https://example.com/angular-portfolio'}],
 [{'links': 'https://example.com/ml-python-portfolio'},
  {'links': 'https://example.com/devops-portfolio'}],
 [{'links': 'https://example.com/ml-python-portfolio'},
  {'links': 'https://example.com/ios-ar-portfolio'}]]

In [15]:
prompt_email = PromptTemplate.from_template(
        """
        ### JOB DESCRIPTION:
        {job_description}
        
        ### INSTRUCTION:
        You are Nirmallya, a business development executive at TCS. TCS is an AI & Software Consulting company dedicated to facilitating
        the seamless integration of business processes through automated tools. 
        Over our experience, we have empowered numerous enterprises with tailored solutions, fostering scalability, 
        process optimization, cost reduction, and heightened overall efficiency. 
        Your job is to write a cold email to the client regarding the job mentioned above describing the capability of TCS
        in fulfilling their needs.
        Also add the most relevant ones from the following links to showcase TCS's portfolio: {link_list}
        Remember you are Nirmallya, BDE at TCS. 
        Do not provide a preamble.
        ### EMAIL (NO PREAMBLE):
        
        """
        )

chain_email = prompt_email | llm
res = chain_email.invoke({"job_description": str(job), "link_list": links})
print(res.content)

Subject: Expert Support for Enterprise Web Application - TCS Solutions

Dear Hiring Manager,

I came across the job description for a Software Engineer - Engineering Support (L2) role, and I believe TCS can provide the ideal candidate to support your enterprise web application. Our team of experts has extensive experience in troubleshooting and determining root causes for technical escalations, ensuring seamless integration of business processes.

At TCS, we have a proven track record of empowering numerous enterprises with tailored solutions, fostering scalability, process optimization, cost reduction, and heightened overall efficiency. Our team is well-equipped to handle medium to high-level complexity support inquiries, providing quick and intelligent responses to production issues.

Our expertise in writing basic to medium complex SQL queries for Oracle/MSSQL databases, Unix Shell scripting, and debugging Java/.NET web applications aligns perfectly with your requirements. We also p